In [1]:
!pip install sentence_transformers faiss-cpu 
!pip install --upgrade python-docx


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 5.2 MB/s eta 0:00:00


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from sentence_transformers import SentenceTransformer
import torch
import faiss
import numpy as np
from docx import Document
import time
import kagglehub 

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3.5-mini-instruct",
    device_map=device,
    torch_dtype="auto",
    cache_dir="./micro-chat",
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct", cache_dir="./micro-chat-T")
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Load the sentence transformer model for embedding generation
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', cache_folder="./embed-model")



config.json:   0%|          | 0.00/3.45k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-mini-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-mini-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
# FAISS index to store embeddings for vector search
dimension = 384  # Dimensionality of the embeddings from 'all-MiniLM-L6-v2'
index = faiss.IndexFlatL2(dimension)
vector_text_mapping = []

def group_sentences(sentences, group_size=15):
    grouped = [' '.join(sentences[i:i+group_size]) for i in range(0, len(sentences), group_size)]
    return grouped


def extract_text_from_docx(file):
    """Extract text from a Word document directly from memory."""
    doc = Document(file)
    full_text = []
    for paragraph in doc.paragraphs:
        full_text.append(paragraph.text)
    return "\n".join(full_text)

def store_text_in_vector_db(text):
    """Generate embeddings for text and store them in the vector database."""
    global index, vector_text_mapping
    sentences = text.split(".")
    sentences = group_sentences(sentences)
    embeddings = embedding_model.encode(sentences, convert_to_tensor=False)
    embeddings = np.array(embeddings)
    index.add(embeddings)
    vector_text_mapping.extend(sentences)

def retrieve_relevant_text(query, top_k=3):
    """Retrieve the most relevant text chunks based on the query."""
    global index, vector_text_mapping
    query_embedding = embedding_model.encode([query], convert_to_tensor=False)
    query_embedding = np.array(query_embedding).astype(np.float32)
    if index.ntotal == 0:
        print("FAISS index is empty. No document has been indexed.")
    distances, indices = index.search(query_embedding, top_k)
    print(f"Distances: {distances}")
    print(f"Indices: {indices}")

    if len(distances[0]) == 0 or np.all(distances[0] >= 1.3):
        return None

    retrieved_text = [vector_text_mapping[i] for i in indices[0]]
    return " ".join(retrieved_text)


In [5]:
file = kagglehub.dataset_download('vrmedicalllm/testing2', path='unjit.docx')
topic = "Packages  and  interfaces"
question_type = "Mcqs"
number_questions = 6

In [6]:
try:
    print(file,topic,question_type,number_questions)
    document_text = ""
    
    # Check if a Word document is provided
    if file and topic:
        document_text = extract_text_from_docx(file)
        store_text_in_vector_db(document_text)
        query = f"{topic}"
        relevant_text = retrieve_relevant_text(query)
    else:
        print({"generated_questions": "Please provide both  topic and a Word document."})
    if relevant_text is None:
            print({"generated_questions": f"No data found regarding the topic:{topic} in the provided document."})
    else:
        start_time = time.time()
        print(relevant_text)
        # Generate questions using the language model
        messages = [
            {"role": "system", "content": f"You are a helpful AI assistant who generates {number_questions} {question_type} questions from given text."},
            {"role": "user", "content": f"Using the following '{relevant_text}' , generate {number_questions} {question_type} questions on the topic '{topic}'"},
        ]
        generation_args = {
            "max_new_tokens": 500,
            "return_full_text": False,
            "temperature": 0.0,
            "do_sample": False,
        }
        output = pipe(messages, **generation_args)

        end_time = time.time()
        elapsed_time = end_time - start_time

        if output and len(output) > 0 and 'generated_text' in output[0]:
            print({
                "generated_questions": output[0]['generated_text'],
                "time_taken": f"{elapsed_time:.2f} seconds"
            })
        else:
            print({"error": "No output generated by the model."})

except Exception as e:
    print({"error": str(e)})


/kaggle/input/testing2/unjit.docx Packages  and  interfaces Mcqs 6


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


Distances: [[0.87597513 0.9443189  0.97419286]]
Indices: [[ 0 14 13]]
UNIT-2


Packages Packages  and  interfaces  are  two  of  the  basic  components  of  a  Java
program  In general, a Java source file can contain any (or all) of the following four
internal parts:
*	A single package statement (optional)
*	Any number of import statements (optional)
*	A single public class declaration (required)
*	Any number of classes private to the package (optional)

Java  provides  a  mechanism  for  partitioning  the  class  name  space  into  more
manageable chunks  This mechanism is the package  The package is both a naming and a
visibility  control  mechanism   You  can  define  classes  inside  a  package  that  are  not
accessible by code outside that package  You can also define class members that are only
exposed to other members of the same package 

Defining a Package:

*	Creating  a  package  is  quite  easy:  simply  include  a  package  command  as  the  first
statement in a Java sour